In [1]:
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import csv
import time

In [2]:
base_url = "http://api.coingecko.com/api/v3"

<h1>Get crypto list</h1>

In [3]:
request_url = "/coins/markets"

params_dict = {"vs_currency" : "USD",
              "per_page" : 250,
              "page" : 1}

In [4]:
response = requests.get(base_url + request_url, params= params_dict)
if response.status_code != 200:
    print("Error")
data = response.json()

In [5]:
list_cryptos = []
for _dict in data:
    list_cryptos.append(_dict['id'])

In [6]:
list_cryptos

['bitcoin',
 'ethereum',
 'binancecoin',
 'ripple',
 'tether',
 'cardano',
 'dogecoin',
 'polkadot',
 'uniswap',
 'litecoin',
 'bitcoin-cash',
 'chainlink',
 'usd-coin',
 'vechain',
 'stellar',
 'solana',
 'theta-token',
 'filecoin',
 'tron',
 'wrapped-bitcoin',
 'binance-usd',
 'monero',
 'neo',
 'pancakeswap-token',
 'terra-luna',
 'klay-token',
 'eos',
 'iota',
 'bitcoin-cash-sv',
 'aave',
 'okb',
 'cosmos',
 'matic-network',
 'crypto-com-chain',
 'bittorrent-2',
 'ftx-token',
 'ethereum-classic',
 'compound-usd-coin',
 'tezos',
 'algorand',
 'huobi-token',
 'avalanche-2',
 'compound-ether',
 'maker',
 'dai',
 'kusama',
 'thorchain',
 'compound-governance-token',
 'cdai',
 'dash',
 'elrond-erd-2',
 'nem',
 'chiliz',
 'holotoken',
 'havven',
 'zcash',
 'enjincoin',
 'decred',
 'zilliqa',
 'hedera-hashgraph',
 'blockstack',
 'leo-token',
 'waves',
 'sushi',
 'celsius-degree-token',
 'nexo',
 'near',
 'amp-token',
 'the-graph',
 'terrausd',
 'decentraland',
 'fantom',
 'digibyte',
 'si

<h1>Get timeseries marketcap and volume data for a list of tokens, for the last year</h1>

In [26]:
wait_time = 60
with open('targets_lists/coin_list.csv', newline=None) as f:
    reader = csv.reader(f)
    coin_list=list(reader)

In [47]:
start_date = datetime.datetime(2020,1,1)
end_date = datetime.datetime(2021,4,30)
params_dict = {"vs_currency" : "usd",
              "from" : datetime.datetime.timestamp(start_date),
              "to" : datetime.datetime.timestamp(end_date)}

In [48]:
request_url = f"/coins/{coin_list[0][0]}/market_chart/range"
response = requests.get(base_url + request_url, params = params_dict)

In [49]:
if response.status_code != 200:
    print(f"API error on {coin_list[0][0]}")
else:
    coin_data = response.json()
    coin_marketcap_data_df = pd.DataFrame(data = coin_data["market_caps"], index=None, columns=["Date", coin_list[0][0]])
    coin_volume_data_df = pd.DataFrame(data = coin_data["total_volumes"], index=None, columns=["Date", coin_list[0][0]])
    coin_price_data_df = pd.DataFrame(data = coin_data["prices"], index=None, columns=["Date", coin_list[0][0]])
    print(f"Successfully saved {coin_list[0][0]} data")

Successfully saved yearn-finance data


In [42]:
if len(coin_list[0]) > 0:
    count = 0
    for coin_id in coin_list[0][1:]:
        request_url = f"/coins/{coin_id}/market_chart/range"
        response = requests.get(base_url + request_url, params = params_dict)
        if response.status_code != 200:
            print(f"API error on {coin_id}")
        else:
            coin_data = response.json()
            coin_mc = [day[1] for day in coin_data["market_caps"]]
            coin_v = [day[1] for day in coin_data["total_volumes"]]
            coin_p = [day[1] for day in coin_data["prices"]]
            coin_marketcap_data_df[[f'{coin_id}']] = pd.DataFrame(coin_mc, index=None)
            coin_volume_data_df[[f'{coin_id}']] = pd.DataFrame(coin_v, index=None)
            coin_price_data_df[[f'{coin_id}']] = pd.DataFrame(coin_p, index=None)
            print(f"Successfully saved {coin_id} data")
        count += 1
        if count%25 == 0:
            f"Wainting for {wait_time} seconds"
            time.sleep(60)
        
coin_marketcap_data_df.to_csv(f"coin_data/coin_marketcap_data.csv", index = False)
coin_volume_data_df.to_csv(f"coin_data/coin_volume_data.csv", index = False)
coin_price_data_df.to_csv(f"coin_data/coin_price_data.csv", index = False)

Successfully saved bitcoin data
Successfully saved ethereum data
Successfully saved binancecoin data
Successfully saved huobi-token data


<h1>Get timeseries volume data for a list of exchanges, for the last year</h1>

In [3]:
with open('targets_lists/exchange_list.csv', newline=None) as f:
    reader = csv.reader(f)
    exchange_list = list(reader)

In [4]:
params_dict = {'days' : 365}

In [13]:
request_url = f"/exchanges/{exchange_list[0][0]}/volume_chart"
response = requests.get(base_url + request_url, params = params_dict)
if response.status_code != 200:
    print(f"API error on {exchange_list[0][0]}")
else:
    exchange_data = response.json()
    exchange_volume_data_df = pd.DataFrame(data = exchange_data, index=None, columns=["Date", exchange_list[0][0]])
    print(f"Successfully saved {exchange_list[0][0]} data")

Successfully saved binance data


In [24]:
request_url = f"/exchanges/{exchange_list[0][1]}/volume_chart"
response = requests.get(base_url + request_url, params = params_dict)
if response.status_code != 200:
    print(f"API error on {exchange_list[0][1]}")
else:
    exchange_data = response.json()
    temp_exchange_df = pd.DataFrame(exchange_data, columns = ['Date', f'{exchange_list[0][1]}'])
    exchange_volume_data_df = exchange_volume_data_df.merge(right = temp_exchange_df, on = 'Date', how = "left")

In [25]:
exchange_volume_data_df.head()

,Date,binance,kraken
0,1.588664e+12,184343.966018427273615570081864484452332509430...,23165.5756558552143636985912455752229679517868...
1,1.588751e+12,154146.629453794782392602183956712659932066684...,19442.9058913852456033495341481623293917890653...
2,1.588837e+12,227711.444082259656861410307238396884131979132...,32120.9360784747146891000941166190785854535081...
3,1.588924e+12,296630.607041383664802599426083733596483228639...,47058.2059166307932521395192699829401101363270...
4,1.589010e+12,222630.405755479304024111016093793255721911069...,24629.2701956428829572401579142160231676237197...


In [23]:
temp_exchange_df.head()


,Date,kraken
0,1.588664e+12,23165.5756558552143636985912455752229679517868...
1,1.588751e+12,19442.9058913852456033495341481623293917890653...
2,1.588837e+12,32120.9360784747146891000941166190785854535081...
3,1.588924e+12,47058.2059166307932521395192699829401101363270...
4,1.589010e+12,24629.2701956428829572401579142160231676237197...


In [45]:
if len(exchange_list[0]) > 0:
    for exchange_id in exchange_list[0][1:]:
        request_url = f"/exchanges/{exchange_id}/volume_chart"
        response = requests.get(base_url + request_url, params = params_dict)
        if response.status_code != 200:
            print(f"API error on {exchange_id}")
        else:
            exchange_data = response.json()
            exchange_v = [day[1] for day in exchange_data]
            exchange_volume_data_df[[f'{exchange_id}']] = pd.DataFrame(exchange_v, index=None)
            print(f"Successfully saved {exchange_id} data")

exchange_volume_data_df.to_csv(f"exchange_data/exchange_volume_data.csv", index = False)

Successfully saved binance data
Successfully saved huobi data
Successfully saved kraken data


<h1>Scrashpad for Dash exploration</h1>

In [4]:
coin_marketcap_df = pd.read_csv('coin_data/coin_marketcap_data.csv')

In [5]:
coin_marketcap_df

,Date,bitcoin,ethereum,binancecoin,huobi-token
0,1587427200000,1.257246e+11,1.889468e+10,2.228118e+09,8.696864e+08
1,1587513600000,1.254731e+11,1.888330e+10,2.232531e+09,8.610005e+08
2,1587600000000,1.303986e+11,2.016527e+10,2.319817e+09,8.953018e+08
3,1587686400000,1.354148e+11,2.037161e+10,2.353230e+09,9.049613e+08
4,1587772800000,1.374937e+11,2.075671e+10,2.352744e+09,9.264940e+08
...,...,...,...,...,...
360,1618531200000,1.179562e+12,2.905920e+11,8.388906e+10,3.783728e+09
361,1618617600000,1.149603e+12,2.806176e+11,7.913832e+10,3.653192e+09
362,1618704000000,1.125592e+12,2.710603e+11,8.014656e+10,3.391717e+09
363,1618790400000,1.053618e+12,2.593291e+11,7.437436e+10,3.114921e+09


In [10]:
coin_marketcap_df['Date'] = pd.to_datetime(arg=coin_marketcap_df['Date'], unit='ms')

In [12]:
coin_marketcap_df[['Date', 'bitcoin']]

,Date,bitcoin
0,2020-04-21,1.257246e+11
1,2020-04-22,1.254731e+11
2,2020-04-23,1.303986e+11
3,2020-04-24,1.354148e+11
4,2020-04-25,1.374937e+11
...,...,...
360,2021-04-16,1.179562e+12
361,2021-04-17,1.149603e+12
362,2021-04-18,1.125592e+12
363,2021-04-19,1.053618e+12
